In [1]:
import sys
sys.path.append('..')

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import VarianceThreshold

from common import load_forest_fires

# Feature Selection

## Resources

sklearn docs - [Feature selection](https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection)

Why giving your algorithm ALL THE FEATURES does not always work - Thomas Huijskens - [youtube](https://youtu.be/JsArBz46_3s)

[Automated Feature Engineering and Selection in Python](https://www.youtube.com/watch?v=4-4pKPv9lJ4)

## Why select features

- colinearity
- reduces noise (+ overfitting)
- more interpretable
- train models quicker
- train models better

Adding features is an exponential cost!
- curse of dimensionality
- model needs to understand the new feature in the context of every other feature

What makes a good feature selection algorithm
- remove low information features
- reduce overlap between features


## Three categories of feature selection

1. Wrappers
- assess performance by performance of a model
- new model for each set of features -> expensive

2. Filter
- only consider statistics of the data (correlation, mutual infomation, variance thresholding)
- ignore interaction with learning algorithm

3. Embedded
- combinations of wrapping & filters
- feature selection as part of the model construction process


In [2]:
ds = load_forest_fires()

x = ds.loc[:, ['FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain']]
y = ds.loc[:, 'area']

ds.describe()

For more information, read [Cortez and Morais, 2007].
        1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
        2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
        3. month - month of the year: 'jan' to 'dec'
        4. day - day of the week: 'mon' to 'sun'
        5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
        6. DMC - DMC index from the FWI system: 1.1 to 291.3
        7. DC - DC index from the FWI system: 7.9 to 860.6
        8. ISI - ISI index from the FWI system: 0.0 to 56.10
        9. temp - temperature in Celsius degrees: 2.2 to 33.30
        10. RH - relative humidity in %: 15.0 to 100
        11. wind - wind speed in km/h: 0.40 to 9.40
        12. rain - outside rain in mm/m2 : 0.0 to 6.4
        13. area - the burned area of the forest (in ha): 0.00 to 1090.84
    (this output variable is very skewed towards 0.0, thus it may make
    sense to model with the logarithm transform).')

data.shape = (517, 

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
count,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000
mean,4.669246,4.299807,90.644681,110.872340,547.940039,9.021663,18.889168,44.288201,4.017602,0.021663,12.847292
std,2.313778,1.229900,5.520111,64.046482,248.066192,4.559477,5.806625,16.317469,1.791653,0.295959,63.655818
min,1.000000,2.000000,18.700000,1.100000,7.900000,0.000000,2.200000,15.000000,0.400000,0.000000,0.000000
25%,3.000000,4.000000,90.200000,68.600000,437.700000,6.500000,15.500000,33.000000,2.700000,0.000000,0.000000
50%,4.000000,4.000000,91.600000,108.300000,664.200000,8.400000,19.300000,42.000000,4.000000,0.000000,0.520000
75%,7.000000,5.000000,92.900000,142.400000,713.900000,10.800000,22.800000,53.000000,4.900000,0.000000,6.570000
max,9.000000,9.000000,96.200000,291.300000,860.600000,56.100000,33.300000,100.000000,9.400000,6.400000,1090.840000


In [3]:
ds.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


In [4]:
ds.columns

Index(['X', 'Y', 'month', 'day', 'FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH',
       'wind', 'rain', 'area'],
      dtype='object')

## Variance selection

Based only on the feature
- no information about the target

In [5]:
sel = VarianceThreshold(threshold=(0.8))
sel.fit_transform(x)

x.columns[sel.get_support()]

Index(['FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind'], dtype='object')

## Univariate feature selection

Selecting features in isolation, based on statistical relationship to the target

[sklearn docs](https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection)

In [6]:
k = 6
selector = SelectKBest(mutual_info_regression, k=k)
features = selector.fit_transform(x, y)

features

array([[ 86.2,  26.2,  94.3,   5.1,   8.2,  51. ],
       [ 90.6,  35.4, 669.1,   6.7,  18. ,  33. ],
       [ 90.6,  43.7, 686.9,   6.7,  14.6,  33. ],
       ...,
       [ 81.6,  56.7, 665.6,   1.9,  21.2,  70. ],
       [ 94.4, 146. , 614.7,  11.3,  25.6,  42. ],
       [ 79.5,   3. , 106.7,   1.1,  11.8,  31. ]])

In [7]:
x.columns[selector.get_support()]

Index(['FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH'], dtype='object')

In [8]:
for score, f in zip(selector.scores_, x.columns):
    print(score, f)

0.020358542159004678 FFMC
0.02236153422733267 DMC
0.05392775567421548 DC
0.041739459265855317 ISI
0.014392199556597873 temp
0.05710959909858371 RH
0.0 wind
0.0028231715364901433 rain


## SelectFromModel

Requires an interpretable model
- coefficients in linear regression
- feature importances

Select features based on a threshold

In [9]:
mdl = ExtraTreesRegressor(n_estimators=50)
mdl.fit(x, y)
model = SelectFromModel(mdl, prefit=True, threshold='mean')
x_new = model.transform(x)
x.columns[model.get_support()]

Index(['DMC', 'temp', 'RH', 'wind'], dtype='object')